In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Vaibhav\\Desktop\\Course\\12_project_1\\Project_01\\research'

In [3]:
os.chdir("../")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [5]:
from src.datascience_project.constants import *
from src.datascience_project.utils.common import read_yaml, create_directory

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directory([self.config.artifacts_root])
    
    def get_model_triner_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directory([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )

        return model_trainer_config

In [19]:
import pandas as pd
import os 
from src.datascience_project import logger
from sklearn.linear_model import ElasticNet
import joblib

In [20]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_X = train_data.drop([self.config.target_column], axis=1)
        test_X = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_X, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [23]:
try:
    config = ConfigurationManager()
    model_train_config = config.get_model_triner_config()
    model_train = ModelTrainer(config=model_train_config)
    model_train.train()
except Exception as e:
    raise e

[2025-05-04 16:01:43,560: INFO: common: yaml file config\config.yaml loaded successfully]
[2025-05-04 16:01:43,563: INFO: common: yaml file params.yaml loaded successfully]
[2025-05-04 16:01:43,566: INFO: common: yaml file schema.yaml loaded successfully]
[2025-05-04 16:01:43,568: INFO: common: created directory at: artifacts]
[2025-05-04 16:01:43,569: INFO: common: created directory at: artifacts/model_trainer]
